In [ ]:
import torch

import arviz as az

from autoemulate.simulations.double_pendulum import DoublePendulum
from autoemulate.core.compare import AutoEmulate
from autoemulate.calibration.bayes import BayesianCalibration
from autoemulate.emulators import GaussianProcess
from autoemulate.transforms import PCATransform


In [ ]:
simulator = DoublePendulum(n_time_points=100)
x = simulator.sample_inputs(500, random_seed=42)
y = simulator.forward_batch(x)


In [ ]:
import matplotlib.pyplot as plt

# Plot theta1 for first 10 samples
plt.figure(figsize=(14, 8))

# Plot theta1 time series for 10 samples
plt.subplot(2, 1, 1)
for i in range(5):
    theta1_series = y[i, :simulator.n_time_points]
    # Get parameter values for this sample
    m1, m2, l1, l2, theta1_0, theta2_0 = x[i].numpy()
    label = f'm1={m1:.2f}, m2={m2:.2f},  theta_1={theta1_0:.2f}'
    plt.plot(simulator.time_points, theta1_series, alpha=0.7, label=label)

plt.xlabel('Time (s)')
plt.ylabel('$\\theta_1$ (rad)')
plt.title('Double Pendulum: $\\theta_1$ vs Time (10 samples)')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# Plot theta2 for the same 10 samples
plt.subplot(2, 1, 2)
for i in range(5):
    theta2_series = y[i, simulator.n_time_points:2*simulator.n_time_points]
    # Get parameter values for this sample
    m1, m2, l1, l2, theta1_0, theta2_0 = x[i].numpy()
    label = f'm1={m1:.2f}, m2={m2:.2f}, theta_2={theta2_0:.2f}'
    plt.plot(simulator.time_points, theta2_series, alpha=0.7, label=label)

plt.xlabel('Time (s)')
plt.ylabel('$\\theta_2$ (rad)')
plt.title('Double Pendulum: $\\theta_2$ vs Time (10 samples)')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
y_new = y[:, :simulator.n_time_points].float()
plt.plot(simulator.time_points,y_new[:2, :].T)
y_new.type()



In [ ]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process.kernels import ExpSineSquared
import numpy as np


# Periodic kernel for oscillatory behavior + RBF for chaos
kernel = RBF(length_scale=1.0) * ExpSineSquared(length_scale=1.0, periodicity=2*np.pi) + WhiteKernel()

ae = AutoEmulate(x, y_new, 
    models=[GaussianProcess],
    model_params={"kernel": kernel},
    model_tuning=False,
    y_transforms_list=[[PCATransform(n_components=8)]], 
    log_level="progress_bar"
)

In [ ]:
ae.summarise()

In [ ]:
ae_2 = AutoEmulate(
    x, 
    y_new, 
    models=[GaussianProcess], 
    y_transforms_list=[[PCATransform(n_components=8)]], 
    log_level="progress_bar")

We can verify that the fitted emulator performs well on both the train and test data.

In [ ]:
ae_2.summarise()

In [ ]:
import gpytorch

# Periodic kernel for oscillatory behavior + RBF for chaos
kernel = gpytorch.kernels.RBFKernel() + gpytorch.kernels.PeriodicKernel()

ae_3 = AutoEmulate(x, y_new, 
    models=[GaussianProcess],
    model_tuning=False,
    model_params={"kernel": kernel},
    y_transforms_list=[[PCATransform(n_components=8)]], 
    log_level="progress_bar"
)

In [ ]:
ae_3.summarise()